In [ ]:
import numpy as np
import pandas as pd
import csv
from collections import namedtuple

In [ ]:
# Big boy things

# Seperator used for main columns in database file
delimiter = "|"
# Seperator used for storing event values
event_delimiter = "^"
# Seperator used for storing multiple events
event_list_delimiter = "*"
# Possible affiliations for events
event_affiliations = {"QA":"QAC", "FE":"Fellows", "RA":"RAs", "OT":"Other"}
# Allow events worth more than 2 points
allow_higher_points = False
# Points max (only relevant if the above variable is false)
point_max = 2

# Create our student struct
Student = namedtuple("Student", "first_name last_name student_id unity_id events points")
# And an event struct
Event = namedtuple("Event", "name points affiliation")

In [ ]:
# Import the village points sheet as a data set
data = open("test.csv")

excel = pd.read_excel("september_points.xlsx")

reader = csv.reader(data, delimiter=delimiter)

In [ ]:
def parseEvent(string):
    # Events should be given in the form: "EventName{event_delimiter}NumPoints{event_delimiter}Affiliation"
    # Where: 
    # {event_delimiter} is defined above
    # NumPoints = 1 or 2
    # Affiliation = QC, FE, RA, OT (Qac, Fellows, RAs, Other)
    components = string.split(event_delimiter)
    
    if len(components) != 3:
        print(f"Invalid event string provided (value = \'{string}\')")
        return ["INVALID", 0, "OT"]
    
    name = components[0]
    points = components[1]
    affil = components[2]
        
    # Make sure the points are valid
    try:
        num_points = int(points)
        if not allow_higher_points and num_points > point_max:
            print(f"Point value above maximum, limit set to {point_max}!")
            raise Exception()
            
        if num_points <= 0:
            raise Exception()
            
        points = num_points
    except:
        print(f"Invalid point value provided for event \'{name}\' (value = \'{points}\'), setting to 1!")
        points = 1
    
    # Make sure the affiliation makes sense
    if affil not in event_affiliations:
        print(f"Invalid affiliation provided for event \'{name}\' (value = \'{affil}\'), setting to \'Other\'!")
        affil = "OT"
    
    return Event(name=name, points=points, affiliation=affil)

def parseEventListString(string):
    # This essentially just uses the previous method several times, based on another delimiter
    event_strings = string.split(event_list_delimiter)
    events = ["" for i in range(len(event_strings))]
    for i in range(len(event_strings)):
        events[i] = parseEvent(event_strings[i])

    return events

In [ ]:
def parseStudent(row):
    # Takes in a string of a row, and returns a student struct
    last_name = row[0]
    first_name = row[1]
    student_id = row[2]
    unity_id = row[3]
    events = row[4]
    points = row[5]
    
    return Student(first_name=first_name, last_name=last_name, student_id=student_id, unity_id=unity_id, events=events, points=points)

In [ ]:
# Event tests
single_event_tests = ["Event 1^1^FE", "Event 2^2^RA", "Event 3^3^QA", ""]
for s in single_event_tests:
    print(parseEvent(s))

list_event_test = "*".join(single_event_tests)[0:-1]
print(parseEventListString(list_event_test))

In [ ]:
for row in reader:
    print(parseStudent(row))